# Walkthrough Challenge 4 - Chat with your own data

Duration: 30 minutes

## Overview
- In this challenge, you will start by configuring and initializing the kernel.
- [TODO]

## Prerequisites

- Please ensure that you have completed the [Setup](../setup/setup.ipynb) before starting this challenge.

### Task 1: Configure and Initialize Semantic Kernel

⚠️ Note: You should have already completed all tasks on the [Setup](../setup/setup.ipynb). If you have not, please go back and complete it now.

#### Step 1: Load Semantic Kernel settings

In this step, we will load the Semantic Kernel settings that we created in the [Setup](../setup/setup.ipynb) notebook.

In [5]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Sqlite, 1.0.1-alpha"
#r "nuget: Microsoft.KernelMemory.Core, 0.26.240121.1"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin, 0.26.240121.1"

#!import ../setup/config/Settings.cs
#!import ../setup/config/Utils.cs

Installed Packages Microsoft.KernelMemory.Core, 0.26.240121.1 Microsoft.KernelMemory.SemanticKernelPlugin, 0.26.240121.1 Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.0.1-preview Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha

Error: (27,43): error CS0246: The type or namespace name 'ChatHistory' could not be found (are you missing a using directive or an assembly reference?)

Error: (27,43): error CS0246: The type or namespace name 'ChatHistory' could not be found (are you missing a using directive or an assembly reference?)

#### Step 2: Initialize Semantic Kernel

In [72]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.Sqlite;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.KernelMemory;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../setup/config/settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

### Task 2: Create a chatbot like experience, mainting the context of the conversation 

We will start by creating a `ChatHistory` object to maintain the context of the conversation. This object will be used to store the conversation history, namely the system prompts, user inputs and the assistant responses.

The `ChatHistory` object will be initialized with the system prompt.`

In [48]:
var chatHistory = new ChatHistory("You are an historian, expert in the Seven Wonders of the Ancient World. Be concise and informative.");

The `MessageOutputAsync` method will allow us to print the latest message from the chat history.

In [58]:
await Utils.MessageOutputAsync(chatHistory);

assistant: The Great Pyramid of Giza, also known as the Pyramid of Khufu or the Pyramid of Cheops, is the oldest and largest of the three pyramids on the Giza plateau in Egypt and the only one of the Seven Wonders of the Ancient World that still exists today. It was built as a tomb for the Fourth Dynasty Egyptian Pharaoh Khufu (Cheops in Greek) and was completed around 2560 BCE.

Constructed from millions of limestone blocks, the original height of the pyramid was 146.6 meters (481 feet), but it now stands at 138.8 meters (455 feet) due to the loss of the outer casing stones. The Great Pyramid was the tallest man-made structure in the world for over 3,800 years.

The precise construction techniques used to build the pyramid are still not fully understood, but the process likely involved a large workforce of thousands of laborers and skilled workers. The pyramid's complex interior includes a series of passageways leading to the main burial chamber, which houses a granite sarcophagus.

T

To interact with the chatbot, we will require the `ChatCompletionService`. This will be used to interact with the Large Language Model to generate responses to the user inputs.

After adding the user input to the chat history, we will use the `ChatCompletionService` to generate a response. This response will be added to the chat history and printed to the user.

In [59]:
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 500,
    Temperature = 0.2,
    TopP = 0.5
};

chatHistory.AddUserMessage("Hi, what is the Great Pyramid of Giza?");
await MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(
        chatHistory,
        executionSettings: executionSettings,
        kernel: kernel);

chatHistory.Add(reply);
await MessageOutputAsync(chatHistory);

user: Hi, what is the Great Pyramid of Giza?
------------------------
assistant: The Great Pyramid of Giza is the oldest and most intact of the Seven Wonders of the Ancient World. It was constructed as a tomb for the Egyptian Pharaoh Khufu (also known as Cheops) and is estimated to have been completed around 2560 BCE. The pyramid is located on the Giza Plateau, near modern-day Cairo, Egypt.

Originally standing at 146.6 meters (481 feet) tall, the Great Pyramid was the tallest man-made structure in the world for over 3,800 years. Although it has lost some of its height due to the absence of the outer casing stones, it still stands at approximately 138.8 meters (455 feet) today.

The pyramid is built from approximately 2.3 million blocks of stone, each weighing on average 2.5 tons. Its construction is a marvel of ancient engineering, and its precise alignment with the cardinal points and the stars has intrigued scholars for centuries.

The Great Pyramid consists of three main chambers: 

Let's add the new message to chat history. You will see that the chat model will use the context of the conversation to generate a response.

Take note that chat history counts towards your token usage, so be mindful of the number of messages you send. Strategies such as counting the number of tokens and truncating the history to only include the last few messages can be used to manage token usage.

In [60]:
chatHistory.AddUserMessage("And do you know who built it?");
await MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(
        chatHistory,
        executionSettings: executionSettings,
        kernel: kernel);

chatHistory.Add(reply);
await MessageOutputAsync(chatHistory);

user: And do you know who built it?
------------------------
assistant: The Great Pyramid of Giza was built under the reign of Pharaoh Khufu, also known as Cheops in Greek. He was the second pharaoh of the Fourth Dynasty of ancient Egypt's Old Kingdom. The workforce that constructed the pyramid is believed to have consisted of tens of thousands of skilled laborers, including stonecutters, masons, engineers, and other workers.

Contrary to the long-standing myth that the pyramids were built by slaves, evidence suggests that the laborers were actually well-fed and housed in a nearby temporary city. They were likely organized into work gangs and worked in shifts to construct the pyramid over a period of approximately 20 years.

The planning and resources required for such a massive project indicate a highly organized society with advanced knowledge of architecture, engineering, and mathematics. The laborers were probably a mix of full-time workers and seasonal conscripts who participated 

You can experiment interacting with the chatbot by adding more messages to the chat history and observing the responses.

In [63]:
do
{
    try
    {
        var ask = await InteractiveKernel.GetInputAsync("Ask a question to the assistant: ");
        chatHistory.AddUserMessage(ask);
        await MessageOutputAsync(chatHistory);

        var reply = await chatCompletionService.GetChatMessageContentAsync(
                chatHistory,
                executionSettings: executionSettings,
                kernel: kernel);

        chatHistory.Add(reply);
        await MessageOutputAsync(chatHistory);
    }
    catch (Exception)
    {
        // break the loop if the user cancels the input
        break;
    }
} while (true);


user: who is cleopetra?
------------------------
assistant: Cleopatra, often referred to as Cleopatra VII, was the last active ruler of the Ptolemaic Kingdom of Egypt. She reigned from 51 BCE until her death in 30 BCE. Cleopatra was a member of the Ptolemaic dynasty, a family of Greek origin that ruled Egypt after Alexander the Great's death during the Hellenistic period. The Ptolemies, including Cleopatra, embraced Egyptian traditions, and Cleopatra herself was the first in her line to learn the Egyptian language.

Cleopatra is well-known for her relationships with two prominent Roman figures: Julius Caesar and Mark Antony. Her affair with Julius Caesar helped her solidify her grip on the throne, and after Caesar's assassination, she aligned with Mark Antony, with whom she had a romantic and political alliance. This alliance led to a conflict with the future first emperor of Rome, Octavian (later known as Augustus).

The conflict culminated in the naval battle of Actium in 31 BCE, whe

Error: Input request cancelled

In [84]:
var memorybuilder = new KernelMemoryBuilder();

memorybuilder.WithAzureOpenAITextEmbeddingGeneration(new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.EmbeddingGeneration,
    Endpoint = azureEndpoint,
    Deployment = model
});

memorybuilder.WithAzureOpenAITextGeneration(new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.ChatCompletion,
    Endpoint = azureEndpoint,
    Deployment = model
});

var memory = memorybuilder.Build();


Installed Packages Microsoft.KernelMemory.Core, 0.26.240121.1 Microsoft.KernelMemory.SemanticKernelPlugin, 0.26.240121.1

In [78]:

await memory.ImportWebPageAsync(
    "https://www.ilo.org/ifpdial/information-resources/national-labour-law-profiles/WCMS_158899/lang--en/index.htm");


In [80]:

var response = await memory.AskAsync("how many vacations days do I get in Germany?");

response.Result.Display();

In Germany, the statutory minimum leave entitlement is 24 days per calendar year, not counting Sundays and public holidays. Saturdays are included in the calculation. However, it is common for collective agreements to grant a period of 4 up to 6 weeks of paid leave per calendar year. For employees whose employment relationships fall under collective agreements reached in 1999, 80% of West German employees and 55% of East German employees can claim paid leave of 6 weeks or more.

In [86]:
using Microsoft.KernelMemory;

var memoryPlugin = kernel.ImportPluginFromObject(new MemoryPlugin(memory));

In [122]:
var skPrompt = """
Question to Memory: {{$input}}

Answer from Memory: {{MemoryPlugin.Ask $input}}

If the answer is empty say 'I don't know' otherwise reply with a preview of the answer,
truncated to 15 words. Prefix with one emoji relevant to the content.
""";

var ragFunction = kernel.CreateFunctionFromPrompt(skPrompt);

Console.WriteLine("Semantic Function ready.");

Semantic Function ready.


In [123]:
var answer = await ragFunction.InvokeAsync(kernel, "how many vacations days do I get in Germany?");

Console.WriteLine(answer);

🌴 In Germany, the statutory minimum leave entitlement is 24 days per calendar year, not counting Sundays...


In [124]:
var answer = await ragFunction.InvokeAsync(kernel, "how many vacations days do I get in Portugal?");

Console.WriteLine(answer);

🏖 I don't know
